In [1]:
from cryoS2Sdrop.dataloader import (singleCET_FourierDataset, tom_deconv_tomo, Parallel, delayed)
from tomoSegmentPipeline.utils.common import read_array, write_array
from tomoSegmentPipeline.utils import setup
from cryoS2Sdrop.misc import parse_cet_paths
import os
from glob import glob
from joblib import Parallel, delayed


import torch

PARENT_PATH = setup.PARENT_PATH

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

/home/ubuntu/Thesis/miniconda3/envs/jero/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
samples_path = os.path.join(
    PARENT_PATH, "data/S2SDenoising/model_logs/tomo04/raw_denoising/FourierSamples"
)
samples_path

'/home/ubuntu/Thesis/data/S2SDenoising/model_logs/tomo04/raw_denoising/FourierSamples'

In [9]:
samples_file = os.path.join(samples_path, "singleCET_FourierDataset.samples")
samples_file

'/home/ubuntu/Thesis/data/S2SDenoising/model_logs/tomo04/raw_denoising/FourierSamples/singleCET_FourierDataset.samples'

In [8]:

os.path.exists(samples_file)

True

In [4]:
len(glob("/home/ubuntu/Thesis/data/S2SDenoising/model_logs/tomo04/raw_denoising_pilot/FourierSamples/*.samples"))

1

In [5]:
%%time
samples = torch.load(samples_file)

CPU times: user 132 ms, sys: 39.2 s, total: 39.4 s
Wall time: 16min 23s


In [6]:
samples.shape

torch.Size([20, 1, 464, 928, 928])

In [7]:
samples_small = samples.split(len(samples)//2)
len(samples_small), samples_small[-1].shape

(2, torch.Size([10, 1, 464, 928, 928]))

In [8]:
del samples

In [9]:
import gc
gc.collect()

14

In [10]:
samples_path = os.path.join(
    PARENT_PATH, "data/S2SDenoising/model_logs/tomo04/raw_denoising_pilot/FourierSamples/"
)
samples_path

'/home/ubuntu/Thesis/data/S2SDenoising/model_logs/tomo04/raw_denoising_pilot/FourierSamples/'

In [ ]:
Parallel(n_jobs=2, verbose=10)(
    delayed(torch.save)(sample, os.path.join(samples_path, "singleCET_FourierDataset%i.samples" %i)) for i, sample in enumerate(samples_small))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [22]:
for i, sample in enumerate(samples_small):
    print(i)
    %time torch.save(sample, os.path.join(samples_path, "singleCET_FourierDataset%i.samples" %i))

0
1
2


In [26]:
aux = os.path.join(samples_path, "singleCET_FourierDataset%i.samples" %i)
aux

'/home/ubuntu/Thesis/data/S2SDenoising/model_logs/tomo04/raw_denoising_pilot/FourierSamples/singleCET_FourierDataset2.samples'

In [27]:
torch.load(aux).shape

torch.Size([1, 1, 464, 928, 928])

In [31]:
my_samples = Parallel(n_jobs=3, verbose=10)(
    delayed(torch.load)(os.path.join(samples_path, "singleCET_FourierDataset%i.samples" %i)) for i in range(len(samples_small)))
my_samples = torch.cat(my_samples)
my_samples.shape

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.9min remaining:    0.0s
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.9min finished


RuntimeError: stack expects each tensor to be equal size, but got [2, 1, 464, 928, 928] at entry 0 and [1, 1, 464, 928, 928] at entry 2

torch.Size([5, 1, 464, 928, 928])